<a href="https://colab.research.google.com/github/NguyenIsHere/Google-colab/blob/main/FullToxicDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- PHẦN 0: IMPORT THƯ VIỆN CẦN THIẾT VÀ KẾT NỐI GOOGLE DRIVE ---
from google.colab import files, drive
import pandas as pd
import io
import os
import shutil # Để copy file model
import re

# --- 0.1: Kết nối Google Drive và thiết lập đường dẫn ---
try:
    drive.mount('/content/drive')
    print("Đã kết nối thành công với Google Drive!")

    # Định nghĩa đường dẫn thư mục trên Google Drive
    # Bạn có thể thay đổi 'Colab_Toxic_Comment_Project' thành tên thư mục bạn muốn.
    drive_project_base_path = '/content/drive/My Drive/Colab_Toxic_Comment_Project/'
    drive_datasets_path = os.path.join(drive_project_base_path, 'datasets/')
    drive_models_path = os.path.join(drive_project_base_path, 'models/')

    # Tạo các thư mục trên Drive nếu chúng chưa tồn tại
    os.makedirs(drive_datasets_path, exist_ok=True)
    os.makedirs(drive_models_path, exist_ok=True)
    print(f"Các file dataset sẽ được lưu tại: {drive_datasets_path}")
    print(f"File model sẽ được lưu tại: {drive_models_path}")
    print("Các thư mục đã sẵn sàng trên Google Drive (hoặc đã tồn tại từ trước).")

except Exception as e:
    print(f"Lỗi khi kết nối hoặc tạo thư mục trên Google Drive: {e}")
    print("Vui lòng kiểm tra quyền truy cập hoặc thử mount lại Drive.")
    # Gán giá trị mặc định để code không bị lỗi nếu Drive không mount được,
    # nhưng các thao tác lưu lên Drive sẽ thất bại.
    drive_datasets_path = None
    drive_models_path = None

# --- PHẦN 1: UPLOAD DATASET, LƯU LÊN DRIVE VÀ ĐỌC VÀO PANDAS ---

def save_uploaded_bytes_to_drive(uploaded_data_dict, filename_in_colab, drive_target_folder_path):
    """Lưu nội dung bytes của file đã upload từ Colab vào Google Drive."""
    if not drive_target_folder_path:
        print(f"Lỗi: Đường dẫn Google Drive chưa được thiết lập. Không thể lưu {filename_in_colab}.")
        return False

    if filename_in_colab in uploaded_data_dict:
        file_bytes = uploaded_data_dict[filename_in_colab]
        destination_on_drive = os.path.join(drive_target_folder_path, filename_in_colab)
        try:
            with open(destination_on_drive, 'wb') as f:
                f.write(file_bytes)
            print(f"Đã lưu file '{filename_in_colab}' vào Google Drive tại: {destination_on_drive}")
            return True
        except Exception as e:
            print(f"Lỗi khi lưu file '{filename_in_colab}' vào Google Drive: {e}")
            return False
    else:
        print(f"Lỗi: Không tìm thấy '{filename_in_colab}' trong dictionary các file đã upload.")
        return False

# Upload file train.csv
print("\n--- Tải lên train.csv ---")
print("Hãy chọn file train.csv của bạn:")
uploaded_train_dict = files.upload()
train_csv_filename = 'train.csv'
if train_csv_filename in uploaded_train_dict:
    if drive_datasets_path: # Chỉ lưu nếu đường dẫn Drive hợp lệ
        save_uploaded_bytes_to_drive(uploaded_train_dict, train_csv_filename, drive_datasets_path)
else:
    print(f"Cảnh báo: Bạn đã không tải lên file có tên '{train_csv_filename}'.")

# Upload file valid.csv
print("\n--- Tải lên valid.csv ---")
print("Hãy chọn file valid.csv của bạn:")
uploaded_valid_dict = files.upload()
valid_csv_filename = 'valid.csv'
if valid_csv_filename in uploaded_valid_dict:
    if drive_datasets_path:
        save_uploaded_bytes_to_drive(uploaded_valid_dict, valid_csv_filename, drive_datasets_path)
else:
    print(f"Cảnh báo: Bạn đã không tải lên file có tên '{valid_csv_filename}'.")

# Upload file test.csv
print("\n--- Tải lên test.csv ---")
print("Hãy chọn file test.csv của bạn:")
uploaded_test_dict = files.upload()
test_csv_filename = 'test.csv'
if test_csv_filename in uploaded_test_dict:
    if drive_datasets_path:
        save_uploaded_bytes_to_drive(uploaded_test_dict, test_csv_filename, drive_datasets_path)
else:
    print(f"Cảnh báo: Bạn đã không tải lên file có tên '{test_csv_filename}'.")

# Đọc file CSV vào pandas DataFrame
print("\n--- Đọc file CSV vào Pandas DataFrame ---")
df_train, df_valid, df_test = None, None, None # Khởi tạo
try:
    if train_csv_filename in uploaded_train_dict:
        df_train = pd.read_csv(io.BytesIO(uploaded_train_dict[train_csv_filename]), encoding="utf-8")
        print(f"\nĐã đọc xong {train_csv_filename}.")
        # print("Thông tin df_train:")
        # df_train.info()
        # print("\n5 dòng đầu của df_train:")
        # print(df_train.head())
    else:
        print(f"'{train_csv_filename}' không được tải lên, không thể đọc vào DataFrame.")

    if valid_csv_filename in uploaded_valid_dict:
        df_valid = pd.read_csv(io.BytesIO(uploaded_valid_dict[valid_csv_filename]), encoding="utf-8")
        print(f"Đã đọc xong {valid_csv_filename}.")
    else:
        print(f"'{valid_csv_filename}' không được tải lên, không thể đọc vào DataFrame.")

    if test_csv_filename in uploaded_test_dict:
        df_test = pd.read_csv(io.BytesIO(uploaded_test_dict[test_csv_filename]), encoding="utf-8")
        print(f"Đã đọc xong {test_csv_filename}.")
    else:
        print(f"'{test_csv_filename}' không được tải lên, không thể đọc vào DataFrame.")

    if df_train is not None:
        print("\nThông tin df_train:")
        df_train.info()
        print("\n5 dòng đầu của df_train:")
        print(df_train.head())
    else:
        print("\nKhông có dữ liệu df_train để hiển thị.")

except KeyError as e:
    print(
        f"Lỗi KeyError: Không tìm thấy file {e} trong các file đã upload. Hãy chắc chắn bạn đã upload đúng file và tên file chính xác."
    )
except Exception as e:
    print(f"Có lỗi xảy ra khi đọc file CSV: {e}")


# --- PHẦN 2: TIỀN XỬ LÝ DỮ LIỆU ---
print("\n--- Tiền xử lý dữ liệu ---")
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^\w\sÀ-ỹ]', '', text)
    text = text.strip()
    return text

# Chỉ thực hiện nếu DataFrames đã được tạo
if df_train is not None and df_valid is not None and df_test is not None:
    if 'Comment' in df_train.columns and 'Comment' in df_valid.columns and 'Comment' in df_test.columns:
        df_train['cleaned_comment'] = df_train['Comment'].apply(preprocess_text)
        df_valid['cleaned_comment'] = df_valid['Comment'].apply(preprocess_text)
        df_test['cleaned_comment'] = df_test['Comment'].apply(preprocess_text)
        print("\nĐã tiền xử lý xong cột Comment.")
        print(df_train[['Comment', 'cleaned_comment']].head())

        # Chuẩn bị dữ liệu cho mô hình
        X_train = df_train['cleaned_comment'].fillna('')
        y_train = df_train['Toxicity']
        X_valid = df_valid['cleaned_comment'].fillna('')
        y_valid = df_valid['Toxicity']
        X_test = df_test['cleaned_comment'].fillna('')
        y_test = df_test['Toxicity']

        data_ready_for_training = True
    else:
        print("Lỗi: Không tìm thấy cột 'Comment' trong một hoặc nhiều DataFrame.")
        data_ready_for_training = False
else:
    print("Lỗi: Một hoặc nhiều DataFrame (train, valid, test) chưa được khởi tạo do lỗi đọc file. Không thể tiền xử lý.")
    data_ready_for_training = False

# --- PHẦN 3: CÀI ĐẶT THƯ VIỆN VÀ HUẤN LUYỆN MODEL (CHỈ CHẠY NẾU DATA SẴN SÀNG) ---
if data_ready_for_training:
    print("\n--- Cài đặt thư viện Transformers và Underthesea ---")
    !pip install transformers -q
    !pip install underthesea -q
    print("Cài đặt thư viện xong.")

    import torch
    from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
    from torch.optim import AdamW
    from torch.utils.data import DataLoader, Dataset
    from sklearn.metrics import classification_report, accuracy_score
    import numpy as np

    # --- 3.1. Định nghĩa Dataset class ---
    class ToxicDataset(Dataset):
        def __init__(self, texts, labels, tokenizer, max_len):
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, item):
            text = str(self.texts[item])
            label = self.labels[item]

            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=False,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt',
            )

            return {
                'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(label, dtype=torch.long)
            }

    # --- 3.2. Thiết lập các tham số ---
    MODEL_NAME = 'vinai/phobert-base'
    MAX_LEN = 128
    BATCH_SIZE = 16
    EPOCHS = 3
    LEARNING_RATE = 2e-5

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nSử dụng thiết bị: {device}")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # --- 3.3. Tạo DataLoaders ---
    def create_data_loader(df_texts, df_labels, tokenizer, max_len, batch_size):
        texts_list = df_texts.tolist() if isinstance(df_texts, pd.Series) else df_texts
        labels_list = df_labels.tolist() if isinstance(df_labels, pd.Series) else df_labels

        ds = ToxicDataset(
            texts=texts_list,
            labels=labels_list,
            tokenizer=tokenizer,
            max_len=max_len
        )
        return DataLoader(ds, batch_size=batch_size, num_workers=2)

    train_data_loader = create_data_loader(X_train, y_train.values, tokenizer, MAX_LEN, BATCH_SIZE)
    valid_data_loader = create_data_loader(X_valid, y_valid.values, tokenizer, MAX_LEN, BATCH_SIZE)
    test_data_loader = create_data_loader(X_test, y_test.values, tokenizer, MAX_LEN, BATCH_SIZE)

    # --- 3.4. Định nghĩa mô hình (Classifier trên nền PhoBERT) ---
    class PhoBERTClassifier(torch.nn.Module):
        def __init__(self, n_classes):
            super(PhoBERTClassifier, self).__init__()
            self.bert = AutoModel.from_pretrained(MODEL_NAME)
            self.drop = torch.nn.Dropout(p=0.3)
            if hasattr(self.bert.config, 'pooler_fc_size') and self.bert.config.pooler_fc_size is not None:
                 self.out = torch.nn.Linear(self.bert.config.pooler_fc_size, n_classes)
            elif hasattr(self.bert.config, 'hidden_size'):
                 self.out = torch.nn.Linear(self.bert.config.hidden_size, n_classes)
            else:
                raise AttributeError("Không tìm thấy hidden_size hoặc pooler_fc_size trong config của model.")

        def forward(self, input_ids, attention_mask):
            outputs = self.bert(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                pooled_output = outputs.pooler_output
            else:
                last_hidden_state = outputs.last_hidden_state
                pooled_output = last_hidden_state[:, 0, :]

            output = self.drop(pooled_output)
            return self.out(output)

    N_CLASSES = 2
    model_phobert = PhoBERTClassifier(N_CLASSES).to(device)

    # --- 3.5. Định nghĩa hàm tối ưu, scheduler, loss function ---
    optimizer = AdamW(model_phobert.parameters(), lr=LEARNING_RATE)
    total_steps = len(train_data_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
    loss_fn = torch.nn.CrossEntropyLoss().to(device)

    # --- 3.6. Vòng lặp Huấn luyện ---
    def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
        model = model.train()
        losses = []
        correct_predictions = 0
        for batch_idx, d in enumerate(data_loader):
            if batch_idx % 100 == 0:
                print(f"  Training batch {batch_idx}/{len(data_loader)}")
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        return correct_predictions.double() / n_examples, np.mean(losses)

    # --- 3.7. Vòng lặp Đánh giá ---
    def eval_model(model, data_loader, loss_fn, device, n_examples):
        model = model.eval()
        losses = []
        correct_predictions = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch_idx, d in enumerate(data_loader):
                if batch_idx % 100 == 0:
                    print(f"  Evaluating batch {batch_idx}/{len(data_loader)}")
                input_ids = d["input_ids"].to(device)
                attention_mask = d["attention_mask"].to(device)
                labels = d["labels"].to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                _, preds = torch.max(outputs, dim=1)
                loss = loss_fn(outputs, labels)
                correct_predictions += torch.sum(preds == labels)
                losses.append(loss.item())
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        return correct_predictions.double() / n_examples, np.mean(losses), all_preds, all_labels

    # --- 3.8. Chạy Huấn luyện và Đánh giá ---
    print("\n--- Bắt đầu huấn luyện mô hình PhoBERT... ---")
    history = {'train_acc': [], 'train_loss': [], 'val_acc': [], 'val_loss': []}
    best_accuracy = 0
    colab_model_filename = 'best_phobert_model.pth' # Tên file model sẽ lưu trong Colab

    for epoch in range(EPOCHS):
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 10)
        train_acc, train_loss = train_epoch(
            model_phobert, train_data_loader, loss_fn, optimizer, device, scheduler, len(X_train)
        )
        print(f'Train loss {train_loss:.4f} accuracy {train_acc:.4f}')
        val_acc, val_loss, _, _ = eval_model(
            model_phobert, valid_data_loader, loss_fn, device, len(X_valid)
        )
        print(f'Val   loss {val_loss:.4f} accuracy {val_acc:.4f}')
        print()
        history['train_acc'].append(train_acc.item() if hasattr(train_acc, 'item') else train_acc)
        history['train_loss'].append(train_loss)
        history['val_acc'].append(val_acc.item() if hasattr(val_acc, 'item') else val_acc)
        history['val_loss'].append(val_loss)

        if val_acc > best_accuracy:
            torch.save(model_phobert, colab_model_filename) # Lưu model tốt nhất vào Colab runtime
            best_accuracy = val_acc
            print(f"Đã lưu model tốt nhất (val_acc: {best_accuracy:.4f}) vào '{colab_model_filename}' trong Colab.")

    print("--- Huấn luyện xong! ---")

    # --- 3.9. Lưu model TỐT NHẤT từ Colab runtime lên Google Drive ---
    if drive_models_path: # Chỉ lưu nếu đường dẫn Drive hợp lệ
        path_to_best_model_in_colab = f'/content/{colab_model_filename}'
        destination_best_model_on_drive = os.path.join(drive_models_path, colab_model_filename)
        if os.path.exists(path_to_best_model_in_colab):
            try:
                shutil.copy(path_to_best_model_in_colab, destination_best_model_on_drive)
                print(f"Đã lưu model tốt nhất '{colab_model_filename}' vào Google Drive tại: {destination_best_model_on_drive}")
            except Exception as e:
                print(f"Lỗi khi lưu model '{colab_model_filename}' vào Google Drive: {e}")
        else:
            print(f"Lỗi: Không tìm thấy file model tốt nhất '{path_to_best_model_in_colab}' trong Colab runtime để lưu lên Drive.")
    else:
        print("Cảnh báo: Đường dẫn Google Drive chưa được thiết lập. Model tốt nhất không được lưu lên Drive.")


    # --- 3.10. Đánh giá trên tập Test với model tốt nhất đã lưu ---
    print("\n--- Đánh giá trên tập test với mô hình PhoBERT tốt nhất ---")
    # Nạp lại model tốt nhất từ file đã lưu trong Colab (để đảm bảo dùng đúng model best)
    path_to_best_model_in_colab = f'/content/{colab_model_filename}'
    if os.path.exists(path_to_best_model_in_colab):
        best_model_loaded = torch.load(path_to_best_model_in_colab, weights_only=False) # Sửa lỗi UnpicklingError
        best_model_loaded = best_model_loaded.to(device)

        test_acc, _, test_preds, test_labels = eval_model(
            best_model_loaded, test_data_loader, loss_fn, device, len(X_test)
        )
        print(f"Test Accuracy: {test_acc:.4f}")
        print(classification_report(test_labels, test_preds, target_names=['Non-Toxic', 'Toxic']))
    else:
        print(f"Không tìm thấy file model '{path_to_best_model_in_colab}' để thực hiện đánh giá trên tập test.")
else:
    print("\n--- Quá trình huấn luyện bị bỏ qua do dữ liệu đầu vào chưa sẵn sàng. ---")

Mounted at /content/drive
Đã kết nối thành công với Google Drive!
Các file dataset sẽ được lưu tại: /content/drive/My Drive/Colab_Toxic_Comment_Project/datasets/
File model sẽ được lưu tại: /content/drive/My Drive/Colab_Toxic_Comment_Project/models/
Các thư mục đã sẵn sàng trên Google Drive (hoặc đã tồn tại từ trước).

--- Tải lên train.csv ---
Hãy chọn file train.csv của bạn:


Saving train.csv to train.csv
Đã lưu file 'train.csv' vào Google Drive tại: /content/drive/My Drive/Colab_Toxic_Comment_Project/datasets/train.csv

--- Tải lên valid.csv ---
Hãy chọn file valid.csv của bạn:


Saving valid.csv to valid.csv
Đã lưu file 'valid.csv' vào Google Drive tại: /content/drive/My Drive/Colab_Toxic_Comment_Project/datasets/valid.csv

--- Tải lên test.csv ---
Hãy chọn file test.csv của bạn:


Saving test.csv to test.csv
Đã lưu file 'test.csv' vào Google Drive tại: /content/drive/My Drive/Colab_Toxic_Comment_Project/datasets/test.csv

--- Đọc file CSV vào Pandas DataFrame ---

Đã đọc xong train.csv.
Đã đọc xong valid.csv.
Đã đọc xong test.csv.

Thông tin df_train:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        7000 non-null   int64 
 1   Comment           7000 non-null   object
 2   Constructiveness  7000 non-null   int64 
 3   Toxicity          7000 non-null   int64 
 4   Title             7000 non-null   object
 5   Topic             7000 non-null   object
dtypes: int64(3), object(3)
memory usage: 328.3+ KB

5 dòng đầu của df_train:
   Unnamed: 0                                            Comment  \
0        6326                               Thật tuyệt vời...!!!   
1        7835  mỹ đã tuột dốc quá nh

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]


--- Bắt đầu huấn luyện mô hình PhoBERT... ---
Epoch 1/3
----------
  Training batch 0/438
  Training batch 100/438
  Training batch 200/438
  Training batch 300/438
  Training batch 400/438
Train loss 0.3130 accuracy 0.8921
  Evaluating batch 0/125
  Evaluating batch 100/125
Val   loss 0.2749 accuracy 0.9025

Đã lưu model tốt nhất (val_acc: 0.9025) vào 'best_phobert_model.pth' trong Colab.
Epoch 2/3
----------
  Training batch 0/438
  Training batch 100/438
  Training batch 200/438
  Training batch 300/438
  Training batch 400/438
Train loss 0.2362 accuracy 0.9160
  Evaluating batch 0/125
  Evaluating batch 100/125
Val   loss 0.2975 accuracy 0.9055

Đã lưu model tốt nhất (val_acc: 0.9055) vào 'best_phobert_model.pth' trong Colab.
Epoch 3/3
----------
  Training batch 0/438
  Training batch 100/438
  Training batch 200/438
  Training batch 300/438
  Training batch 400/438
Train loss 0.1842 accuracy 0.9397
  Evaluating batch 0/125
  Evaluating batch 100/125
Val   loss 0.3285 accuracy 0.